# Week 2: Image Processing Basics — Lab Practice

**Topics:** Pixels, Point Processing & Histograms

This notebook accompanies the Week 2 lecture slides. We will load images as NumPy arrays, perform point processing operations, compute histograms, and explore color spaces — all hands-on.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb

%matplotlib inline

### Environment setup

This notebook works both **locally** and on **Google Colab**.
- **Local**: images are loaded from the repository's `images/` folder.
- **Colab**: images are automatically downloaded from GitHub on first run.

In [ ]:
import os, urllib.request

# Detect Google Colab
IN_COLAB = "google.colab" in str(get_ipython()) if hasattr(__builtins__, "__IPYTHON__") else False

# Image paths
REPO_URL = "https://raw.githubusercontent.com/HyeongminLEE/image-processing-tutorial/main"
IMAGE_DIR = "images"
IMAGE_NAME = "parrots_square.jpg"

if IN_COLAB:
    os.makedirs(IMAGE_DIR, exist_ok=True)
    url = f"{REPO_URL}/{IMAGE_DIR}/{IMAGE_NAME}"
    dest = os.path.join(IMAGE_DIR, IMAGE_NAME)
    if not os.path.exists(dest):
        print(f"Downloading {IMAGE_NAME} ...")
        urllib.request.urlretrieve(url, dest)
        print("Done.")
    IMAGE_PATH = dest
else:
    # Local: image lives in the repo root's images/ folder
    IMAGE_PATH = os.path.join("..", IMAGE_DIR, IMAGE_NAME)

GRAY_PATH = "parrots_gray.jpg"  # will be created in Section 1

print(f"Image path: {IMAGE_PATH}")
print(f"Running on: {'Google Colab' if IN_COLAB else 'Local'}")

---
## 1. Loading and Exploring Images

We use **PIL** (`Pillow`) to open image files and immediately convert them to NumPy arrays. This gives us full control through array indexing and arithmetic.

Key functions:
- `Image.open(path)` — returns a PIL Image object
- `np.array(pil_img)` — converts it to a NumPy ndarray
- `.shape`, `.dtype`, `.min()`, `.max()` — basic array inspection

In [ ]:
img_color = np.array(Image.open(IMAGE_PATH))

print(f"Shape : {img_color.shape}")   # (H, W, C)
print(f"Dtype : {img_color.dtype}")   # uint8
print(f"Min   : {img_color.min()}")
print(f"Max   : {img_color.max()}")

In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(img_color)
plt.axis("off")
plt.title("Color image (RGB)")
plt.tight_layout()
plt.show()

### Creating and saving a grayscale version

`Image.convert('L')` uses the ITU-R 601-2 luma formula:

$$L = 0.299 R + 0.587 G + 0.114 B$$

We save it to disk so we can load it directly in later sections.

In [ ]:
img_gray = np.array(Image.open(IMAGE_PATH).convert("L"))

print(f"Shape : {img_gray.shape}")   # (H, W) — no channel dimension
print(f"Dtype : {img_gray.dtype}")

# Save for later use
Image.fromarray(img_gray).save(GRAY_PATH)
print(f"Saved grayscale image to {GRAY_PATH}")

In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(img_gray, cmap="gray", vmin=0, vmax=255)
plt.axis("off")
plt.title("Grayscale image")
plt.tight_layout()
plt.show()

### Accessing individual pixels

- Color pixel → a 1D array of 3 values `[R, G, B]`
- Grayscale pixel → a single scalar value

In [ ]:
row, col = 100, 200

print(f"Color pixel at ({row}, {col}): {img_color[row, col]}")
print(f"Gray  pixel at ({row}, {col}): {img_gray[row, col]}")

---
## 2. Channel Extraction

An RGB image has 3 channels stored along axis 2. We can slice them out individually.

When displaying a single channel with `imshow`, use `cmap='gray'` — otherwise matplotlib applies a colormap that can be misleading.

In [ ]:
r_channel = img_color[:, :, 0]
g_channel = img_color[:, :, 1]
b_channel = img_color[:, :, 2]

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(img_color)
axes[0].set_title("Original")

for ax, ch, name in zip(axes[1:], [r_channel, g_channel, b_channel], ["Red", "Green", "Blue"]):
    ax.imshow(ch, cmap="gray", vmin=0, vmax=255)
    ax.set_title(f"{name} channel")

for ax in axes:
    ax.axis("off")

plt.tight_layout()
plt.show()

### Exercises

**Exercise 2.1:** Create three *color* images that isolate each channel: a red-only image (G and B set to 0), a green-only image, and a blue-only image. Display all four (original + 3 filtered) in a 1×4 grid.

*Hint:* Start with `np.zeros_like(img_color)` and copy the desired channel into it.

In [ ]:
# YOUR CODE HERE

---
## 3. Flip, Rotate, and Crop

NumPy slicing gives us geometric transforms for free:

| Operation | Code |
|---|---|
| Vertical flip | `img[::-1]` |
| Horizontal flip | `img[:, ::-1]` |
| 90° rotation | `np.rot90(img)` |
| Crop | `img[y1:y2, x1:x2]` |

In [ ]:
v_flip = img_color[::-1]
h_flip = img_color[:, ::-1]
rot90  = np.rot90(img_color)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for ax, im, title in zip(axes,
                          [img_color, h_flip, v_flip, rot90],
                          ["Original", "H-flip", "V-flip", "Rot 90°"]):
    ax.imshow(im)
    ax.set_title(title)
    ax.axis("off")

plt.tight_layout()
plt.show()

### Cropping

Remember the coordinate order: `img[row_start:row_end, col_start:col_end]`, i.e., **y first, x second**.

In [ ]:
H, W = img_color.shape[:2]
cropped = img_color[H//4 : 3*H//4, W//4 : 3*W//4]

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(img_color)
axes[0].set_title(f"Original ({H}×{W})")
axes[1].imshow(cropped)
axes[1].set_title(f"Center crop ({cropped.shape[0]}×{cropped.shape[1]})")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Exercises

**Exercise 3.1:** Rotate the image by 180° using **only slicing** (do not use `np.rot90`).

In [ ]:
# YOUR CODE HERE

**Exercise 3.2:** Crop a region of your choice, then create a 1×2 figure showing the crop and its horizontally-flipped version.

In [ ]:
# YOUR CODE HERE

---
## 4. Point Processing: Arithmetic Operations

Point processing transforms each pixel **independently**: $g(x,y) = T[f(x,y)]$.

The simplest transforms are arithmetic: add, subtract, multiply.

> **Critical:** Images are stored as `uint8` (0–255). Arithmetic can overflow or underflow. Always **cast to a wider type first**, then **clip** back to [0, 255].

In [ ]:
gray = np.array(Image.open(GRAY_PATH))
print(f"Shape: {gray.shape}, dtype: {gray.dtype}")

### Brightness: Add / Subtract

In [ ]:
bright = np.clip(gray.astype(np.int16) + 80, 0, 255).astype(np.uint8)
dark   = np.clip(gray.astype(np.int16) - 80, 0, 255).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for ax, im, title in zip(axes, [dark, gray, bright], ["Darker (−80)", "Original", "Brighter (+80)"]):
    ax.imshow(im, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.tight_layout()
plt.show()

### Contrast: Multiply

In [ ]:
high_contrast = np.clip(gray.astype(np.float64) * 2.0, 0, 255).astype(np.uint8)
low_contrast  = (gray.astype(np.float64) * 0.5).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for ax, im, title in zip(axes,
                          [low_contrast, gray, high_contrast],
                          ["×0.5 (low contrast)", "Original", "×2.0 (high contrast)"]):
    ax.imshow(im, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.tight_layout()
plt.show()

### Complement / Negative

In [ ]:
negative = 255 - gray  # uint8 subtraction is safe here since 255 >= any pixel

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(gray, cmap="gray", vmin=0, vmax=255)
axes[0].set_title("Original")
axes[1].imshow(negative, cmap="gray", vmin=0, vmax=255)
axes[1].set_title("Negative (255 − f)")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Why casting matters: uint8 overflow demo

`uint8` arithmetic **wraps around** modulo 256. For example, `200 + 100 = 44` (not 300). This produces garbage results.

In [ ]:
# BAD: direct uint8 addition
bad_result = gray + np.uint8(200)

# GOOD: cast first, clip, cast back
good_result = np.clip(gray.astype(np.int16) + 200, 0, 255).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(gray, cmap="gray", vmin=0, vmax=255)
axes[0].set_title("Original")
axes[1].imshow(bad_result, cmap="gray", vmin=0, vmax=255)
axes[1].set_title("uint8 overflow (WRONG)")
axes[2].imshow(good_result, cmap="gray", vmin=0, vmax=255)
axes[2].set_title("With clipping (CORRECT)")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Visualizing transfer functions

A transfer function maps each input intensity to an output intensity. We can plot them on a 256×256 grid.

In [ ]:
x = np.arange(256)

plt.figure(figsize=(5, 5))
plt.plot(x, x, "k--", label="Identity")
plt.plot(x, np.clip(x + 80, 0, 255), label="+80")
plt.plot(x, np.clip(x - 80, 0, 255), label="−80")
plt.plot(x, np.clip(x * 2, 0, 255), label="×2")
plt.plot(x, (x * 0.5).astype(int), label="×0.5")
plt.xlabel("Input intensity")
plt.ylabel("Output intensity")
plt.title("Transfer functions")
plt.legend()
plt.tight_layout()
plt.show()

### Exercises

**Exercise 4.1:** Create a "high-contrast negative" of the grayscale image: first invert it (`255 − f`), then multiply by 1.5 with proper clipping. Display the original and the result side by side.

In [ ]:
# YOUR CODE HERE

**Exercise 4.2:** Apply brightness adjustment (+60) to the **color** image (all three channels). Make sure to handle overflow correctly. Display before and after.

In [ ]:
# YOUR CODE HERE

---
## 5. Gamma Correction

Gamma correction applies a **power-law** transform:

$$g = 255 \left(\frac{f}{255}\right)^\gamma$$

- $\gamma < 1$: brightens dark regions (expands shadows)
- $\gamma = 1$: identity (no change)
- $\gamma > 1$: darkens the image (compresses shadows)

We normalize to [0, 1], apply the power, then scale back to [0, 255].

In [ ]:
x = np.arange(256)

plt.figure(figsize=(5, 5))
plt.plot(x, x, "k--", label="γ = 1.0 (identity)")
for gamma in [0.3, 0.5, 2.0, 3.0]:
    y = 255 * (x / 255.0) ** gamma
    plt.plot(x, y, label=f"γ = {gamma}")
plt.xlabel("Input")
plt.ylabel("Output")
plt.title("Gamma curves")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def apply_gamma(img, gamma):
    """Apply gamma correction to a uint8 image (grayscale or color)."""
    return (255 * (img / 255.0) ** gamma).astype(np.uint8)

In [ ]:
gamma_low  = apply_gamma(gray, 0.4)
gamma_high = apply_gamma(gray, 2.5)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for ax, im, title in zip(axes,
                          [gamma_low, gray, gamma_high],
                          ["γ = 0.4 (brighten)", "Original", "γ = 2.5 (darken)"]):
    ax.imshow(im, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.tight_layout()
plt.show()

### Gamma on a color image

The same function works element-wise on all three channels.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, im, title in zip(axes,
                          [apply_gamma(img_color, 0.4), img_color, apply_gamma(img_color, 2.5)],
                          ["γ = 0.4", "Original", "γ = 2.5"]):
    ax.imshow(im)
    ax.set_title(title)
    ax.axis("off")
plt.tight_layout()
plt.show()

### Exercises

**Exercise 5.1:** Apply 5 different gamma values (0.2, 0.5, 1.0, 2.0, 5.0) to the grayscale image. Display all 5 results in a single 1×5 grid with their gamma values as titles.

In [ ]:
# YOUR CODE HERE

---
## 6. Lookup Tables (LUT)

Any point transform $T: [0,255] \to [0,255]$ can be stored as an array of 256 entries. Applying it to an image is then a single **fancy-indexing** operation:

```python
T = np.array([...])  # shape (256,), dtype uint8
result = T[img]       # each pixel used as an index
```

This is extremely fast because there are no per-pixel arithmetic operations at runtime — just table lookups.

### Threshold LUT

In [ ]:
T_thresh = np.zeros(256, dtype=np.uint8)
T_thresh[128:] = 255

thresh_result = T_thresh[gray]

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(gray, cmap="gray", vmin=0, vmax=255)
axes[0].set_title("Original")
axes[1].plot(np.arange(256), T_thresh)
axes[1].set_title("Threshold LUT")
axes[1].set_xlabel("Input")
axes[1].set_ylabel("Output")
axes[2].imshow(thresh_result, cmap="gray", vmin=0, vmax=255)
axes[2].set_title("Result")
axes[0].axis("off")
axes[2].axis("off")
plt.tight_layout()
plt.show()

### More LUT examples: Solarize & Posterize

- **Solarize**: Invert pixels above a threshold → creates an "X" shaped transfer function
- **Posterize** (4 levels): Quantize 256 grey levels into just 4 steps

In [ ]:
x = np.arange(256)

# Solarize: invert pixels above 128
T_solar = x.copy().astype(np.uint8)
T_solar[128:] = 255 - T_solar[128:]

# Posterize: 4 levels (0, 85, 170, 255)
T_poster = np.zeros(256, dtype=np.uint8)
T_poster[0:64]    = 0
T_poster[64:128]  = 85
T_poster[128:192] = 170
T_poster[192:256] = 255

fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Row 0: transfer functions
for ax, lut, name in zip(axes[0], [T_thresh, T_solar, T_poster],
                          ["Threshold", "Solarize", "Posterize (4 levels)"]):
    ax.plot(x, lut)
    ax.plot(x, x, "k--", alpha=0.3)
    ax.set_title(name)
    ax.set_xlabel("Input")
    ax.set_ylabel("Output")

# Row 1: results
for ax, lut in zip(axes[1], [T_thresh, T_solar, T_poster]):
    ax.imshow(lut[gray], cmap="gray", vmin=0, vmax=255)
    ax.axis("off")

plt.tight_layout()
plt.show()

### Gamma as a LUT

Any formula-based transform can also be pre-computed as a LUT. This is especially useful when applying the same transform to many images.

In [ ]:
gamma_val = 0.4
T_gamma = (255 * (np.arange(256) / 255.0) ** gamma_val).astype(np.uint8)

lut_result    = T_gamma[gray]
direct_result = apply_gamma(gray, gamma_val)

# Verify they are identical
print(f"LUT vs direct — identical? {np.array_equal(lut_result, direct_result)}")

### Exercises

**Exercise 6.1:** Build a custom piecewise LUT:
- Input 0–63 → Output 0
- Input 64–191 → Linear ramp from 0 to 255
- Input 192–255 → Output 255

Plot the transfer function and apply it to the grayscale image.

*Hint:* `np.linspace(0, 255, 128).astype(np.uint8)` gives you 128 evenly spaced values.

In [ ]:
# YOUR CODE HERE

**Exercise 6.2:** Create a "warm tone" effect on the **color** image by applying different LUTs per channel: slightly boost red (gamma 0.8), keep green unchanged (identity), and slightly reduce blue (gamma 1.3). Display before and after.

In [ ]:
# YOUR CODE HERE

---
## 7. Histograms

A histogram counts how many pixels have each intensity value (0–255). It reveals the **contrast characteristics** of an image at a glance.

Key function: `np.histogram(img.ravel(), bins=256, range=[0, 256])`

- `.ravel()` flattens the image to 1D (required by `np.histogram`)
- `bins=256` gives one bin per grey level
- Returns `(hist, bin_edges)` — we typically ignore `bin_edges`

In [ ]:
hist, bins = np.histogram(gray.ravel(), 256, [0, 256])

plt.figure(figsize=(8, 3))
plt.fill_between(np.arange(256), hist, alpha=0.7)
plt.xlabel("Pixel intensity")
plt.ylabel("Count")
plt.title("Histogram of grayscale image")
plt.tight_layout()
plt.show()

### Histogram shapes tell a story

Let's create dark, bright, and low-contrast versions and compare their histograms.

In [ ]:
dark_img    = np.clip(gray.astype(np.float64) * 0.4, 0, 255).astype(np.uint8)
bright_img  = np.clip(gray.astype(np.int16) + 100, 0, 255).astype(np.uint8)
low_cont    = np.clip(gray.astype(np.float64) * 0.3 + 100, 0, 255).astype(np.uint8)

images = [dark_img, gray, bright_img, low_cont]
titles = ["Dark", "Original", "Bright", "Low contrast"]

fig, axes = plt.subplots(2, 4, figsize=(16, 6))

for i, (im, title) in enumerate(zip(images, titles)):
    axes[0, i].imshow(im, cmap="gray", vmin=0, vmax=255)
    axes[0, i].set_title(title)
    axes[0, i].axis("off")

    h, _ = np.histogram(im.ravel(), 256, [0, 256])
    axes[1, i].fill_between(np.arange(256), h, alpha=0.7)
    axes[1, i].set_xlim([0, 255])

plt.tight_layout()
plt.show()

### RGB histogram overlay

For color images, we can plot the histogram of each channel on the same axes.

In [ ]:
plt.figure(figsize=(8, 3))
for ch, color in zip(range(3), ["red", "green", "blue"]):
    h, _ = np.histogram(img_color[:, :, ch].ravel(), 256, [0, 256])
    plt.fill_between(np.arange(256), h, alpha=0.35, color=color, label=color.capitalize())
plt.xlabel("Pixel intensity")
plt.ylabel("Count")
plt.title("RGB histogram")
plt.legend()
plt.tight_layout()
plt.show()

### Exercises

**Exercise 7.1:** Compute the histogram of the grayscale image, then find and print:
1. The most frequent pixel value (mode)
2. The range `[lo, hi]` that contains 90% of all pixels (use the CDF: find the 5th and 95th percentile bins)


In [ ]:
# YOUR CODE HERE

**Exercise 7.2:** Plot the histograms of the original grayscale image and its negative side by side. What is the relationship between the two? Write your answer as a comment.

In [ ]:
# YOUR CODE HERE

---
## 8. Histogram Stretching

Stretching linearly maps the actual intensity range $[f_{\min}, f_{\max}]$ to the full range $[0, 255]$:

$$g = \frac{f - f_{\min}}{f_{\max} - f_{\min}} \times 255$$

This is effective when the image has a genuinely narrow intensity range.

In [ ]:
# Create a low-contrast image
low = np.clip(gray.astype(np.float64) * 0.4 + 80, 0, 255).astype(np.uint8)
print(f"Low-contrast range: [{low.min()}, {low.max()}]")

# Stretch
f_min, f_max = low.min(), low.max()
stretched = ((low.astype(np.float64) - f_min) / (f_max - f_min) * 255).astype(np.uint8)
print(f"Stretched range:    [{stretched.min()}, {stretched.max()}]")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 7))

axes[0, 0].imshow(low, cmap="gray", vmin=0, vmax=255)
axes[0, 0].set_title("Low contrast")
axes[0, 0].axis("off")

axes[0, 1].imshow(stretched, cmap="gray", vmin=0, vmax=255)
axes[0, 1].set_title("Stretched")
axes[0, 1].axis("off")

for ax, im in zip(axes[1], [low, stretched]):
    h, _ = np.histogram(im.ravel(), 256, [0, 256])
    ax.fill_between(np.arange(256), h, alpha=0.7)
    ax.set_xlim([0, 255])

plt.tight_layout()
plt.show()

### When stretching fails

If even one pixel is 0 and another is 255, then $f_{\min}=0, f_{\max}=255$, and the formula simplifies to $g = f$ — no change.

In [ ]:
# Inject outlier pixels into the low-contrast image
low_with_outliers = low.copy()
low_with_outliers[0, 0] = 0
low_with_outliers[0, 1] = 255

f_min2, f_max2 = low_with_outliers.min(), low_with_outliers.max()
stretched2 = ((low_with_outliers.astype(np.float64) - f_min2) / (f_max2 - f_min2) * 255).astype(np.uint8)

print(f"Range with outliers: [{f_min2}, {f_max2}]")
print(f"Stretching produces identical image? {np.array_equal(low_with_outliers, stretched2)}")

### Exercises

**Exercise 8.1:** Implement **percentile-based stretching**: use the 2nd and 98th percentiles as boundaries instead of the absolute min/max. This makes stretching robust to outliers. Apply it to `low_with_outliers` and compare with basic stretching.

*Hint:* `np.percentile(img, 2)` gives the 2nd percentile value.

In [ ]:
# YOUR CODE HERE

---
## 9. Histogram Equalization

Equalization redistributes pixel intensities so the histogram becomes approximately **uniform**. Unlike stretching, it considers the full distribution — not just min/max.

**Algorithm:**
1. Compute histogram $h[k]$
2. Compute CDF: $\text{CDF}[k] = \sum_{j=0}^{k} h[j]$
3. Normalize: $T[k] = \text{round}\left(\frac{\text{CDF}[k] - \text{CDF}_{\min}}{N - \text{CDF}_{\min}} \times 255\right)$
4. Apply as LUT: `result = T[img]`

In [ ]:
# Step-by-step equalization on the low-contrast image
hist_low, _ = np.histogram(low.ravel(), 256, [0, 256])

cdf = hist_low.cumsum()
cdf_normalized = (cdf - cdf.min()) * 255 / (cdf.max() - cdf.min())
cdf_normalized = cdf_normalized.astype(np.uint8)

equalized = cdf_normalized[low]

### Visualizing the CDF as a transfer function

The normalized CDF **is** the LUT used for equalization.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histogram + CDF on twin axes
ax1 = axes[0]
ax2 = ax1.twinx()
ax1.fill_between(np.arange(256), hist_low, alpha=0.5, label="Histogram")
ax2.plot(np.arange(256), cdf / cdf.max(), "r-", label="CDF")
ax1.set_xlabel("Pixel intensity")
ax1.set_ylabel("Count")
ax2.set_ylabel("CDF (normalized)")
ax1.set_title("Histogram and CDF")
ax1.legend(loc="upper left")
ax2.legend(loc="center right")

# Transfer function (the LUT)
axes[1].plot(np.arange(256), cdf_normalized, label="Equalization LUT")
axes[1].plot(np.arange(256), np.arange(256), "k--", alpha=0.3, label="Identity")
axes[1].set_xlabel("Input")
axes[1].set_ylabel("Output")
axes[1].set_title("Transfer function")
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 7))

axes[0, 0].imshow(low, cmap="gray", vmin=0, vmax=255)
axes[0, 0].set_title("Low contrast")
axes[0, 0].axis("off")

axes[0, 1].imshow(equalized, cmap="gray", vmin=0, vmax=255)
axes[0, 1].set_title("Equalized")
axes[0, 1].axis("off")

for ax, im in zip(axes[1], [low, equalized]):
    h, _ = np.histogram(im.ravel(), 256, [0, 256])
    ax.fill_between(np.arange(256), h, alpha=0.7)
    ax.set_xlim([0, 255])

plt.tight_layout()
plt.show()

### Stretching vs Equalization

Let's compare both methods side by side on the same low-contrast input.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for ax, im, title in zip(axes,
                          [low, stretched, equalized],
                          ["Low contrast", "Stretched", "Equalized"]):
    ax.imshow(im, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.tight_layout()
plt.show()

### Exercises

**Exercise 9.1:** Implement histogram equalization from scratch as a function `equalize(img)` that takes a **grayscale** uint8 image and returns the equalized image. Test it on the original grayscale parrot (not the artificial low-contrast version). Display original and equalized images with their histograms in a 2×2 grid.

In [ ]:
# YOUR CODE HERE

---
## 10. HSV Color Space

**HSV** (Hue, Saturation, Value) separates *color* from *intensity*, which is often more intuitive than RGB.

We use `matplotlib.colors.rgb_to_hsv` and `hsv_to_rgb`:
- Input/output are **float arrays in [0, 1]** (so divide by 255 first, multiply by 255 after)
- Hue is in [0, 1] where 0 = red, 1/3 = green, 2/3 = blue, 1 = red again

In [ ]:
hsv = rgb_to_hsv(img_color / 255.0)

h_ch = hsv[:, :, 0]  # Hue        [0, 1]
s_ch = hsv[:, :, 1]  # Saturation [0, 1]
v_ch = hsv[:, :, 2]  # Value      [0, 1]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
axes[0].imshow(img_color)
axes[0].set_title("Original")
axes[1].imshow(h_ch, cmap="hsv", vmin=0, vmax=1)
axes[1].set_title("Hue")
axes[2].imshow(s_ch, cmap="gray", vmin=0, vmax=1)
axes[2].set_title("Saturation")
axes[3].imshow(v_ch, cmap="gray", vmin=0, vmax=1)
axes[3].set_title("Value")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Hue shift

Shifting hue rotates all colors around the color wheel. Since hue is cyclic, we use modulo 1.0.

In [ ]:
hsv_shifted = hsv.copy()
hsv_shifted[:, :, 0] = (hsv_shifted[:, :, 0] + 1/3) % 1.0  # +120 degrees

shifted_rgb = (hsv_to_rgb(hsv_shifted) * 255).astype(np.uint8)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(img_color)
axes[0].set_title("Original")
axes[1].imshow(shifted_rgb)
axes[1].set_title("Hue +120°")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Saturation manipulation

In [ ]:
def adjust_saturation(img_rgb, factor):
    """Multiply saturation by factor, clip to [0, 1]."""
    hsv_tmp = rgb_to_hsv(img_rgb / 255.0)
    hsv_tmp[:, :, 1] = np.clip(hsv_tmp[:, :, 1] * factor, 0, 1)
    return (hsv_to_rgb(hsv_tmp) * 255).astype(np.uint8)

desat = adjust_saturation(img_color, 0.3)
boost = adjust_saturation(img_color, 2.0)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, im, title in zip(axes,
                          [desat, img_color, boost],
                          ["Desaturated (×0.3)", "Original", "Boosted (×2.0)"]):
    ax.imshow(im)
    ax.set_title(title)
    ax.axis("off")
plt.tight_layout()
plt.show()

### Value (brightness) manipulation

In [ ]:
def adjust_value(img_rgb, factor):
    """Multiply Value channel by factor, clip to [0, 1]."""
    hsv_tmp = rgb_to_hsv(img_rgb / 255.0)
    hsv_tmp[:, :, 2] = np.clip(hsv_tmp[:, :, 2] * factor, 0, 1)
    return (hsv_to_rgb(hsv_tmp) * 255).astype(np.uint8)

darker  = adjust_value(img_color, 0.4)
lighter = adjust_value(img_color, 1.5)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, im, title in zip(axes,
                          [darker, img_color, lighter],
                          ["Darker (V×0.4)", "Original", "Lighter (V×1.5)"]):
    ax.imshow(im)
    ax.set_title(title)
    ax.axis("off")
plt.tight_layout()
plt.show()

### Practical application: Equalize only the V channel

Applying histogram equalization to each RGB channel independently can produce color artifacts. A better approach: convert to HSV, equalize only the Value channel (brightness), and convert back. This preserves the original colors.

In [ ]:
# Equalize only the V channel
hsv_eq = rgb_to_hsv(img_color / 255.0)

v_uint8 = (hsv_eq[:, :, 2] * 255).astype(np.uint8)
hist_v, _ = np.histogram(v_uint8.ravel(), 256, [0, 256])
cdf_v = hist_v.cumsum()
cdf_v_norm = (cdf_v - cdf_v.min()) * 255 / (cdf_v.max() - cdf_v.min())
v_equalized = cdf_v_norm[v_uint8].astype(np.uint8)

hsv_eq[:, :, 2] = v_equalized / 255.0
result_v_eq = (hsv_to_rgb(hsv_eq) * 255).astype(np.uint8)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(img_color)
axes[0].set_title("Original")
axes[1].imshow(result_v_eq)
axes[1].set_title("V-channel equalized")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Exercises

**Exercise 10.1:** Extract the V channel from the HSV image, convert it to uint8, and compare it with the grayscale image we created earlier using `Image.convert('L')`. Are they identical? Print whether they match and display both side by side. Explain the difference in a comment.

*Hint:* `Image.convert('L')` uses the luma formula (weighted sum of R, G, B), while V = max(R, G, B).

In [ ]:
# YOUR CODE HERE

**Exercise 10.2:** Create a 2×3 grid showing the original image and 5 hue-shifted versions (+60°, +120°, +180°, +240°, +300°). Label each with its shift angle.

In [ ]:
# YOUR CODE HERE